In [1]:
import os
from ecoscape_layers import (
    LayerGenerator,
    warp,
    RedList,
    generate_resistance_table,
    in_habs,
    default_refinement_method
)

In [6]:
DATA_PATH = "."

In [7]:
REDLIST_KEY = input("IUCN Red List API key: ")
EBIRD_KEY = input("EBird API key: ")

# Set this path to point to the global Jung et al. landcover layer
landcover_fn = os.path.join(DATA_PATH, "inputs", "iucn_habitatclassification_composite_lvl2_ver004.tif")

# Set this path to IUCN's bird range data: http://datazone.birdlife.org/species/requestdis
# You can set this to None if you choose to use only the eBird range maps.
iucn_range_src = os.path.join(DATA_PATH, "inputs", "BOTW.gdb")

# Create IUCN Redlist Object for to fetch habitat data
redlist = RedList(REDLIST_KEY, EBIRD_KEY)

In the following code, we generate the habitat and terrain for the Pileated Woodpecker across the United States.

In [8]:
# REQUIRED INPUTS

species_list = ["pilwoo"]
out_landcover_fn = os.path.join(DATA_PATH, "outputs", "us_terrain.tif")

# OPTIONAL INPUTS

# This projection is applicable to the contiguous United States.
crs = '''
PROJCS["USA_Contiguous_Albers_Equal_Area_Conic",
    GEOGCS["GCS_North_American_1983",
        DATUM["D_North_American_1983",
            SPHEROID["GRS_1980",6378137.0,298.257222101]],
        PRIMEM["Greenwich",0.0],
        UNIT["Degree",0.0174532925199433]],
    PROJECTION["Albers"],
    PARAMETER["False_Easting",0.0],
    PARAMETER["False_Northing",0.0],
    PARAMETER["Central_Meridian",-96.0],
    PARAMETER["Standard_Parallel_1",29.5],
    PARAMETER["Standard_Parallel_2",45.5],
    PARAMETER["Latitude_Of_Origin",37.5],
    UNIT["Meter",1.0]]
'''
resolution = 300
resampling = "near"
bounds = (-2361582.2755981818772852, -1354601.2291806936264038, 2263786.1910862834192812, 1570638.9576217615976930)
padding = 100000

In [20]:
# Process the landcover first.
warp(landcover_fn, out_landcover_fn, crs, resolution, bounds, padding, resampling)

# Initialize layer generator.
layer_generator = LayerGenerator(out_landcover_fn, REDLIST_KEY, EBIRD_KEY)

# Then run the habitat layer generation process for each bird species.
for species_code in species_list:
    habitat_fn = os.path.join(DATA_PATH, "outputs", species_code, "habitat.tif")
    resistance_dict_fn = os.path.join(DATA_PATH, "outputs", species_code, "resistance.csv")
    range_fn = os.path.join(DATA_PATH, "outputs", species_code, "range_map_2022.gpkg")
    range_src = "ebird"

    # get IUCN Redlist Habitat data
    habitat_data = redlist.get_habitat_data(species_code, ebird_code=True if range_src == "ebird" else False)

    # define the refinement methods
    def forest_refinement_method(map_code: int, habitats: dict[int, dict[str, str | bool]]) -> float:
        if in_habs(map_code, ["forest"]):
            return 0.0
        return default_refinement_method(map_code, habitats)

    # create the resistance csv
    generate_resistance_table(habitat_data, resistance_dict_fn, refinement_method=forest_refinement_method)

    # define the overrides for what is current habitat
    cur_hab: list[str | int] = ["forest"]

    # create the habitat layer
    layer_generator.generate_habitat(
        species_code,
        habitat_data,
        habitat_fn,
        range_fn,
        range_src,
        current_hab_overrides=cur_hab,
    )

We now generate habitat and terrain for the Acorn Woodpecker similarly, but for an extent that spans the United States and Mexico.

In [10]:
# REQUIRED INPUTS

species_list = ["acowoo"]
out_landcover_fn = os.path.join(DATA_PATH, "outputs", "us_mexico_terrain.tif")

# OPTIONAL INPUTS

# This projection is modified from the previous one to be used for the US and Mexico.
crs = '''
PROJCS["Albers_US_Mexico",
    GEOGCS["GCS_North_American_1983",
        DATUM["D_North_American_1983",
            SPHEROID["GRS_1980",6378137.0,298.257222101]],
        PRIMEM["Greenwich",0.0],
        UNIT["Degree",0.0174532925199433]],
    PROJECTION["Albers"],
    PARAMETER["False_Easting",0.0],
    PARAMETER["False_Northing",0.0],
    PARAMETER["Central_Meridian",-96.0],
    PARAMETER["Standard_Parallel_1",20.34],
    PARAMETER["Standard_Parallel_2",43.57],
    PARAMETER["Latitude_Of_Origin",31.955],
    UNIT["Meter",1.0]]
'''
resolution = 300
resampling = "near"
bounds = (-2365546.2319482644088566, -1932573.0506286544, 2289506.8540614326484501, 2156960.3359509995207191)
padding = 100000

In [22]:
# Process the landcover first.
warp(landcover_fn, out_landcover_fn, crs, resolution, bounds, padding, resampling)

# Initialize layer generator.
layer_generator = LayerGenerator(out_landcover_fn, REDLIST_KEY, EBIRD_KEY)

# Then run the habitat layer generation process for each bird species.
for species_code in species_list:
    habitat_fn = os.path.join(DATA_PATH, "outputs", species_code, "habitat.tif")
    resistance_dict_fn = os.path.join(DATA_PATH, "outputs", species_code, "resistance.csv")
    range_fn = os.path.join(DATA_PATH, "outputs", species_code, "range_map_2022.gpkg")
    range_src = "ebird"

    # get IUCN Redlist Habitat data
    habitat_data = redlist.get_habitat_data(species_code, ebird_code=True if range_src == "ebird" else False)

    # add custom habitat data for 308
    habitat_data[308] = {
        "code": "3.8",
        "habitat": "Shrubland - Mediterranean-type shrubby vegetation",
        "suitability": True,
        "season": "Resident",
        "majorimportance": True,
    }

    # define the refinement methods
    def forest_refinement_method(map_code: int, habitats: dict[int, dict[str, str | bool]]) -> float:
        if in_habs(map_code, ["forest", 308]):
            return 0.0
        return default_refinement_method(map_code, habitats)

    # create the resistance csv
    generate_resistance_table(habitat_data, resistance_dict_fn, refinement_method=forest_refinement_method)

    # define the overrides for what is current habitat
    cur_hab = ["forest", 308]

    # create the habitat layer
    layer_generator.generate_habitat(
        species_code,
        habitat_data,
        habitat_fn,
        range_fn,
        range_src,
        current_hab_overrides=cur_hab,
    )

Finally, we generate habitats and terrain for several bird species in the Rio de Janeiro region:
- White-bibbed Antbird (whbant4)
- Hooded Berryeater (hoober2)
- Rio de Janeiro Antbird (rdjant1)
- Pin-tailed Manakin (pitman1)
- Saffron Toucanet (saftou2)

Because eBird Status and Trends does not provide range map data for these birds, we use IUCN range map data instead. The IUCN range map data can be requested [here](http://datazone.birdlife.org/species/requestdis).

In [18]:
# REQUIRED INPUTS

species_list = ["whbant4", "hoober2", "rdjant1", "pitman1", "saftou2"] # ebird names

# Convert eBird names to scientific names which are required to use iucn as range_src
from ebird.api import get_taxonomy
for i, species_code in enumerate(species_list):
    species_list[i] = get_taxonomy(species_code)[0]["sciName"]

out_landcover_fn = os.path.join(DATA_PATH, "outputs", "rio_de_janeiro_terrain.tif")

# OPTIONAL INPUTS

# This projection is roughly centered around the Rio de Janeiro region.
crs = '''
PROJCS["Albers_Rio_de_Janeiro",
    GEOGCS["GCS_North_American_1983",
        DATUM["D_North_American_1983",
            SPHEROID["GRS_1980",6378137.0,298.257222101]],
        PRIMEM["Greenwich",0.0],
        UNIT["Degree",0.0174532925199433]],
    PROJECTION["Albers"],
    PARAMETER["False_Easting",0.0],
    PARAMETER["False_Northing",0.0],
    PARAMETER["Central_Meridian",-42.88],
    PARAMETER["Standard_Parallel_1",-24.56],
    PARAMETER["Standard_Parallel_2",-19.56],
    PARAMETER["Latitude_Of_Origin",-22.07],
    UNIT["Meter",1.0]]
'''
resolution = 300
resampling = "near"
bounds = (-205414.97243820442, -144968.97394524177, 199414.99604955647, 144425.33243957412)
padding = 100000

In [24]:
warp(landcover_fn, out_landcover_fn, crs, resolution, bounds, padding, resampling)

# Note that we pass in the IUCN file containing a compilation of bird species ranges in order
# to use the IUCN range maps; eBird doesn't provide range maps for these non-US species.
layer_generator = LayerGenerator(out_landcover_fn, REDLIST_KEY, EBIRD_KEY, iucn_range_src=iucn_range_src)

for species_code in species_list:
    habitat_fn = os.path.join(DATA_PATH, "outputs", species_code, "habitat.tif")
    resistance_dict_fn = os.path.join(DATA_PATH, "outputs", species_code, "resistance.csv")
    range_fn = os.path.join(DATA_PATH, "outputs", species_code, "iucn_range.shp")
    range_src = "iucn"

    # get IUCN Redlist Habitat data
    habitat_data = redlist.get_habitat_data(species_code)

    # define the refinement methods
    def forest_refinement_method(map_code: int, habitats: dict[int, dict[str, str | bool]]) -> float:
        if in_habs(map_code, ["forest"]):
            return 0.0
        return default_refinement_method(map_code, habitats)

    # create the resistance csv
    generate_resistance_table(habitat_data, resistance_dict_fn, refinement_method=forest_refinement_method)

    # define the overrides for what is current habitat
    cur_hab: list[str | int] = ["forest"]

    # create the habitat layer
    layer_generator.generate_habitat(
        species_code,
        habitat_data,
        habitat_fn,
        range_fn,
        range_src,
        current_hab_overrides=cur_hab,
    )